# Hướng dẫn sử dụng Linked Server từ SQL Server sang Oracle

_Lưu ý: Notebook này có thể tải về và mở trong Azure Data Studio để chạy các câu query._

## 1\. Một số thuật ngữ & thông tin cơ bản

1. SQL Server (tên đầy đủ là Microsoft SQL Server), là một hệ thống quản trị cơ sở dữ liệu (relational database management system - RDBMS) phát triển bởi Microsoft.
    
2. Oracle cũng là một RDBMS khác được phát triển bởi Oracle.
    
3. SQL là ngôn ngữ truy vấn dữ liệu, được sử dụng để truy vấn và quản trị dữ liệu lưu trữ trên các RDBMS. cả SQL Server và Oracle đều sử dụng SQL, tuy nhiên mỗi hệ thống có một "phiên bản" SQL riêng: SQL Server sử dụng T-SQL và Oracle sử dụng PL/SQL.
    
4. **SQL Server Management Studio** là ứng dụng để tương tác với SQL Server. **SQL Developer** là ứng dụng để tương tác với Oracle.
    

## 2\. Giới thiệu về Linked Server

### 2.1. Thông tin cơ bản

**Linked Server** sang Oracle được tạo trên máy chủ **172.17.103.40** với thông tin như sau:

- Tên Linked Server: ORASRV
- Data source: 172.17.101.148/MOMPDB

### 2.2. Các cách truy vấn dữ liệu từ Linked Server

#### 2.2.1. Truy vấn trực tiếp

Dữ liệu từ Oracle có thể truy vấn trực tiếp thông qua Linked Server theo cú pháp như sau:

In [1]:
select top 1 * from ORASRV..IT_USER.STG_T24_SAOKE_DEBT_COLLECTION

(1 row affected)

Total execution time: 00:00:02.830

COB_DT,UPD_BY,UPD_TIME,RUN_DATE,Y_SEL_DATE,Y_CONTRACT_REF,LD_ID,CUS_ID,YID_AC,YWK_BAL,Y_SONGAY_QH,Y_SONGAY_QH_T1,Y_SONGAY_QH_T2,Y_SONGAY_QH_T3,Y_SONGAY_QH_T4,Y_SONGAY_QH_T5,Y_KYHAN,Y_KYTRANO,Y_DATE_NEXT,Y_PR_NEXT,Y_IN_NEXT,Y_QH_PR,Y_QH_IN,Y_QH_PE,YAMT_PE_T1,YAMT_PE_T2,YAMT_PE_T3,YAMT_PE_T4,YAMT_PE_T5,Y_QH_PS,YAMT_PS_T1,YAMT_PS_T2,YAMT_PS_T3,YAMT_PS_T4,YAMT_PS_T5,Y_TOTAL_PE,YTOTAL_PE_PS_T1,YTOTAL_PE_PS_T2,YTOTAL_PE_PS_T3,YTOTAL_PE_PS_T4,YTOTAL_PE_PS_T5,YTOTAL_AMT_LD,YTOTAL_AMT_LD_AC,Y_TOTAL_QH,Y_TOTAL_QH_AC,Y_TOTAL_QH_AC_T1,Y_TOTAL_QH_AC_T2,Y_TOTAL_QH_AC_T3,Y_TOTAL_QH_AC_T4,Y_TOTAL_QH_AC_T5,Y_TOTAL_REPAY,Y_TOTAL_REPAY_T1,Y_TOTAL_REPAY_T2,Y_TOTAL_REPAY_T3,Y_TOTAL_REPAY_T4,Y_TOTAL_REPAY_T5,Y_TOTAL_REPAY_AC,Y_TOTAL_REPAY_AC_T1,Y_TOTAL_REPAY_AC_T2,Y_TOTAL_REPAY_AC_T3,Y_TOTAL_REPAY_AC_T4,Y_TOTAL_REPAY_AC_T5,Y_DUNO_CL,CUS_NAME,Y_EARLY_RPY,Y_EARLY_RPY_DATE,YWK_BAL_43,Y_CATEGORY,PRODUCT_TYPE,Y_SUB_NAME,Y_DRAWDOWN_NET_AMT,Y_RFD_AMT
2017-10-04 00:00:00.0000000,JOB_STG_T24_SAOKE_DEBT_COLLECTION_D01,2019-01-15 14:34:46.0000000,2017-10-04 00:00:00.0000000,2017-10-04 00:00:00.0000000,1000117090039557,LD1726410507,8039183,14227760002,NULL,NULL,NULL,NULL,NULL,NULL,NULL,36,1,2017-10-10 00:00:00.0000000,562830,1053555,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1628385,1628385,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,31650000,VO THI TUYET MAI,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


Đây là cách truy vấn đơn giản nhất theo cú pháp của T-SQL. Tuy nhiên cách truy vấn này có nhược điểm là tốc độ truy vấn chậm.

Lý do là bởi khi ta yêu cầu SQL server thực hiện một câu lệnh cho Oracle thông qua Linked Server, SQL Server sẽ lấy ra toàn bộ dữ liệu của cả bảng ở bên Oracle rồi sau đó mới áp đặt các điều kiệu ở WHERE. Giả sử bảng gốc trên Oracle có 10 triệu dòng, trong khi ta chỉ cần query 100 dòng, SQL Server sẽ lấy toàn bộ 10 triệu dòng rồi sau đó mới áp dụng điều kiện lọc để lấy 100 dòng cần thiết.

#### 2.2.2. Truy vấn thông qua OPENQUERY

Có thể hiểu đơn giản việc dùng phương thức truy vấn trực tiếp như ở trên là: Ta để SQL Server tự "dịch" các câu lệnh từ ngôn ngữ T-SQL sang PL/SQL. 

Ngược lại, nếu dùng `OPENQUERY`, ta sẽ tự viết câu lệnh truy vấn theo cú pháp của PL/SQL, rồi chỉ để SQL Server gửi câu lệnh này sang cho Oracle.

Cú pháp của `OPENQUERY` như câu truy vấn ở dưới, trong đó `OPENQUERY` nhận 2 tham số là:

1. Tên Linked Server, trong trường hợp này là **ORASRV** (Viết tắt của Oracle Server), và
2. Câu truy vấn THEO CÚ PHÁP của ORACLE (PL/SQL), nằm trong nháy đơn.

In [2]:
SELECT top 1 * FROM 
OPENQUERY(ORASRV ,'Select * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION' )  


(1 row affected)

Total execution time: 00:00:00.060

COB_DT,UPD_BY,UPD_TIME,RUN_DATE,Y_SEL_DATE,Y_CONTRACT_REF,LD_ID,CUS_ID,YID_AC,YWK_BAL,Y_SONGAY_QH,Y_SONGAY_QH_T1,Y_SONGAY_QH_T2,Y_SONGAY_QH_T3,Y_SONGAY_QH_T4,Y_SONGAY_QH_T5,Y_KYHAN,Y_KYTRANO,Y_DATE_NEXT,Y_PR_NEXT,Y_IN_NEXT,Y_QH_PR,Y_QH_IN,Y_QH_PE,YAMT_PE_T1,YAMT_PE_T2,YAMT_PE_T3,YAMT_PE_T4,YAMT_PE_T5,Y_QH_PS,YAMT_PS_T1,YAMT_PS_T2,YAMT_PS_T3,YAMT_PS_T4,YAMT_PS_T5,Y_TOTAL_PE,YTOTAL_PE_PS_T1,YTOTAL_PE_PS_T2,YTOTAL_PE_PS_T3,YTOTAL_PE_PS_T4,YTOTAL_PE_PS_T5,YTOTAL_AMT_LD,YTOTAL_AMT_LD_AC,Y_TOTAL_QH,Y_TOTAL_QH_AC,Y_TOTAL_QH_AC_T1,Y_TOTAL_QH_AC_T2,Y_TOTAL_QH_AC_T3,Y_TOTAL_QH_AC_T4,Y_TOTAL_QH_AC_T5,Y_TOTAL_REPAY,Y_TOTAL_REPAY_T1,Y_TOTAL_REPAY_T2,Y_TOTAL_REPAY_T3,Y_TOTAL_REPAY_T4,Y_TOTAL_REPAY_T5,Y_TOTAL_REPAY_AC,Y_TOTAL_REPAY_AC_T1,Y_TOTAL_REPAY_AC_T2,Y_TOTAL_REPAY_AC_T3,Y_TOTAL_REPAY_AC_T4,Y_TOTAL_REPAY_AC_T5,Y_DUNO_CL,CUS_NAME,Y_EARLY_RPY,Y_EARLY_RPY_DATE,YWK_BAL_43,Y_CATEGORY,PRODUCT_TYPE,Y_SUB_NAME,Y_DRAWDOWN_NET_AMT,Y_RFD_AMT
2017-10-04 00:00:00.0000000,JOB_STG_T24_SAOKE_DEBT_COLLECTION_D01,2019-01-15 14:34:46.0000000,2017-10-04 00:00:00.0000000,2017-10-04 00:00:00.0000000,1000117090039557,LD1726410507,8039183,14227760002,NULL,NULL,NULL,NULL,NULL,NULL,NULL,36,1,2017-10-10 00:00:00.0000000,562830,1053555,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1628385,1628385,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,31650000,VO THI TUYET MAI,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


Có thể nhận thấy rõ sự khác biệt về tốc độ truy vấn của 2 cách thức với cùng một câu lệnh query ra 10 dòng đầu ở bảng `STG_T24_SAOKE_DEBT_COLLECTION`:

- **Truy vấn trực tiếp**: Total execution time: 00:00:02.830
- **Truy vấn qua OPENQUERY**: Total execution time: 00:00:00.060

Lưu ý rằng ta còn có thể cải thiện tốc độ truy vấn hơn nữa nếu chuyển điều kiện "TOP 10" từ SQL Server vào bên trong câu truy vấn PL/SQL như sau:

In [4]:
-- rownum <= N tương đương với top N
SELECT * FROM 
OPENQUERY(ORASRV ,'Select * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION where rownum <= 1' )

(1 row affected)

Total execution time: 00:00:00.021

COB_DT,UPD_BY,UPD_TIME,RUN_DATE,Y_SEL_DATE,Y_CONTRACT_REF,LD_ID,CUS_ID,YID_AC,YWK_BAL,Y_SONGAY_QH,Y_SONGAY_QH_T1,Y_SONGAY_QH_T2,Y_SONGAY_QH_T3,Y_SONGAY_QH_T4,Y_SONGAY_QH_T5,Y_KYHAN,Y_KYTRANO,Y_DATE_NEXT,Y_PR_NEXT,Y_IN_NEXT,Y_QH_PR,Y_QH_IN,Y_QH_PE,YAMT_PE_T1,YAMT_PE_T2,YAMT_PE_T3,YAMT_PE_T4,YAMT_PE_T5,Y_QH_PS,YAMT_PS_T1,YAMT_PS_T2,YAMT_PS_T3,YAMT_PS_T4,YAMT_PS_T5,Y_TOTAL_PE,YTOTAL_PE_PS_T1,YTOTAL_PE_PS_T2,YTOTAL_PE_PS_T3,YTOTAL_PE_PS_T4,YTOTAL_PE_PS_T5,YTOTAL_AMT_LD,YTOTAL_AMT_LD_AC,Y_TOTAL_QH,Y_TOTAL_QH_AC,Y_TOTAL_QH_AC_T1,Y_TOTAL_QH_AC_T2,Y_TOTAL_QH_AC_T3,Y_TOTAL_QH_AC_T4,Y_TOTAL_QH_AC_T5,Y_TOTAL_REPAY,Y_TOTAL_REPAY_T1,Y_TOTAL_REPAY_T2,Y_TOTAL_REPAY_T3,Y_TOTAL_REPAY_T4,Y_TOTAL_REPAY_T5,Y_TOTAL_REPAY_AC,Y_TOTAL_REPAY_AC_T1,Y_TOTAL_REPAY_AC_T2,Y_TOTAL_REPAY_AC_T3,Y_TOTAL_REPAY_AC_T4,Y_TOTAL_REPAY_AC_T5,Y_DUNO_CL,CUS_NAME,Y_EARLY_RPY,Y_EARLY_RPY_DATE,YWK_BAL_43,Y_CATEGORY,PRODUCT_TYPE,Y_SUB_NAME,Y_DRAWDOWN_NET_AMT,Y_RFD_AMT
2017-10-04 00:00:00.0000000,JOB_STG_T24_SAOKE_DEBT_COLLECTION_D01,2019-01-15 14:34:46.0000000,2017-10-04 00:00:00.0000000,2017-10-04 00:00:00.0000000,1000117090039557,LD1726410507,8039183,14227760002,NULL,NULL,NULL,NULL,NULL,NULL,NULL,36,1,2017-10-10 00:00:00.0000000,562830,1053555,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1628385,1628385,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,1628385,31650000,VO THI TUYET MAI,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


### 2.3. Cách sử dụng OPENQUERY

#### 2.3.1. Áp dụng điều kiện trong OPENQUERY

Có thể thấy rõ ngay một nhược điểm của OPENQUERY, đó là câu query của ORACLE sẽ phải đưa vào trong nháy đơn, tức là việc đưa vào các điều kiện sẽ khó khăn hơn bình thường. Ví dụ:

In [5]:
SELECT top 1 * FROM 
OPENQUERY(ORASRV ,'Select * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION  where Y_SEL_DATE = ''2021-05-23''' )

(1 row affected)

Total execution time: 00:00:00.031

COB_DT,UPD_BY,UPD_TIME,RUN_DATE,Y_SEL_DATE,Y_CONTRACT_REF,LD_ID,CUS_ID,YID_AC,YWK_BAL,Y_SONGAY_QH,Y_SONGAY_QH_T1,Y_SONGAY_QH_T2,Y_SONGAY_QH_T3,Y_SONGAY_QH_T4,Y_SONGAY_QH_T5,Y_KYHAN,Y_KYTRANO,Y_DATE_NEXT,Y_PR_NEXT,Y_IN_NEXT,Y_QH_PR,Y_QH_IN,Y_QH_PE,YAMT_PE_T1,YAMT_PE_T2,YAMT_PE_T3,YAMT_PE_T4,YAMT_PE_T5,Y_QH_PS,YAMT_PS_T1,YAMT_PS_T2,YAMT_PS_T3,YAMT_PS_T4,YAMT_PS_T5,Y_TOTAL_PE,YTOTAL_PE_PS_T1,YTOTAL_PE_PS_T2,YTOTAL_PE_PS_T3,YTOTAL_PE_PS_T4,YTOTAL_PE_PS_T5,YTOTAL_AMT_LD,YTOTAL_AMT_LD_AC,Y_TOTAL_QH,Y_TOTAL_QH_AC,Y_TOTAL_QH_AC_T1,Y_TOTAL_QH_AC_T2,Y_TOTAL_QH_AC_T3,Y_TOTAL_QH_AC_T4,Y_TOTAL_QH_AC_T5,Y_TOTAL_REPAY,Y_TOTAL_REPAY_T1,Y_TOTAL_REPAY_T2,Y_TOTAL_REPAY_T3,Y_TOTAL_REPAY_T4,Y_TOTAL_REPAY_T5,Y_TOTAL_REPAY_AC,Y_TOTAL_REPAY_AC_T1,Y_TOTAL_REPAY_AC_T2,Y_TOTAL_REPAY_AC_T3,Y_TOTAL_REPAY_AC_T4,Y_TOTAL_REPAY_AC_T5,Y_DUNO_CL,CUS_NAME,Y_EARLY_RPY,Y_EARLY_RPY_DATE,YWK_BAL_43,Y_CATEGORY,PRODUCT_TYPE,Y_SUB_NAME,Y_DRAWDOWN_NET_AMT,Y_RFD_AMT
2021-05-23 00:00:00.0000000,JOB_STG_T24_SAOKE_DEBT_COLLECTION_D01,2021-05-24 06:37:50.0000000,2021-05-23 00:00:00.0000000,2021-05-23 00:00:00.0000000,1000320060317241,LD2018102258,1059130,10126023346,180,0,NULL,NULL,NULL,NULL,NULL,27,11,2021-06-11 00:00:00.0000000,986522,687360,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1685882,1685702,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1673882,1673882,1673882,1673882,1673882,1673882,1673702,1673702,1673702,1673702,1673702,1673702,21873268,LE CONG TUOC,NULL,NULL,NULL,21061,CS,CS SY A 37,30000000.00,NULL


Ở trong câu truy vấn trên, điều kiện `WHERE Y_SEL_DATE = ''2021-05-23''` phức tạp hơn bình thường do câu truy vấn nằm trong nháy đơn nên phần ngày `2021-05-23` phải lồng ở trong 2 lần nháy đơn.

Tuy nhiên thông thường khi truy vấn dữ liệu, ta muốn quá trình truy vấn diễn ra tự động và vì thế thường khai các biến (variable) và sử dụng biến này trong các điều kiện truy vấn.

Sử dụng bảng `nguyennqb.dbo.SAOKEDEBT_20210522` ở SQL Server (tương ứng với bảng `STG_T24_SAOKE_DEBT_COLLECTION` ở Oracle, cả 2 đều là sao kê debt), ta có thể truy vấn dữ liệu như sau:

In [6]:
declare @y_sel_date varchar(8) = '20210522'
Select top 1 * from nguyennqb.dbo.SAOKEDEBT_20210522  where [Y SEL DATE] = @y_sel_date

(1 row affected)

Total execution time: 00:00:00.156

Y SEL DATE,Y CONTRACT REF,LD ID,CUS ID,YID AC,YWK BAL,Y SONGAY QH,Y SONGAY QH T1,Y SONGAY QH T2,Y SONGAY QH T3,Y SONGAY QH T4,Y SONGAY QH T5,Y KYHAN,Y KYTRANO,Y DATE NEXT,Y PR NEXT,Y IN NEXT,Y QH PR,Y QH IN,Y QH PE,YAMT PE T1,YAMT PE T2,YAMT PE T3,YAMT PE T4,YAMT PE T5,Y QH PS,YAMT PS T1,YAMT PS T2,YAMT PS T3,YAMT PS T4,YAMT PS T5,Y TOTAL PE,YTOTAL PE PS T1,YTOTAL PE PS T2,YTOTAL PE PS T3,YTOTAL PE PS T4,YTOTAL PE PS T5,YTOTAL AMT LD,YTOTAL AMT LD AC,Y TOTAL QH,Y TOTAL QH AC,Y TOTAL QH AC T1,Y TOTAL QH AC T2,Y TOTAL QH AC T3,Y TOTAL QH AC T4,Y TOTAL QH AC T5,Y TOTAL REPAY,Y TOTAL REPAY T1,Y TOTAL REPAY T2,Y TOTAL REPAY T3,Y TOTAL REPAY T4,Y TOTAL REPAY T5,Y TOTAL REPAY AC,Y TOTAL REPAY AC T1,Y TOTAL REPAY AC T2,Y TOTAL REPAY AC T3,Y TOTAL REPAY AC T4,Y TOTAL REPAY AC T5,Y DUNO CL,CUS NAME,Y EARLY RPY,Y EARLY RPY DATE,YWK BAL 43,Y CATEGORY,PRODUCT TYPE,Y SUB NAME,Y DRAWDOWN NET AMT,Y RFD AMT
20210522,1000320080284178,10127503406,348151,10107037433,0,0,,,,,,36,9,20210608,314958.825,497814,,,,,,,,,,,,,,,0,0,0,0,0,0,824772.825,824772.825,0,,,,,,,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,812772.825,12598353,LE THI KIM TUYEN,,,-10868,HOP DONG HAN MUC,REVOLVING.LOAN,CS RL CS DUO 46.6,18990000,


Để sử dụng biến khi truy vấn bằng OPENQUERY, ta phải làm như sau:

In [11]:
declare @y_sel_date varchar(12) = '2021-05-23'
exec('select top 1 * from OPENQUERY(ORASRV ,''Select * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION  where Y_SEL_DATE = '''''+@y_sel_date+''''''' )' )

(1 row affected)

Total execution time: 00:00:00.021

COB_DT,UPD_BY,UPD_TIME,RUN_DATE,Y_SEL_DATE,Y_CONTRACT_REF,LD_ID,CUS_ID,YID_AC,YWK_BAL,Y_SONGAY_QH,Y_SONGAY_QH_T1,Y_SONGAY_QH_T2,Y_SONGAY_QH_T3,Y_SONGAY_QH_T4,Y_SONGAY_QH_T5,Y_KYHAN,Y_KYTRANO,Y_DATE_NEXT,Y_PR_NEXT,Y_IN_NEXT,Y_QH_PR,Y_QH_IN,Y_QH_PE,YAMT_PE_T1,YAMT_PE_T2,YAMT_PE_T3,YAMT_PE_T4,YAMT_PE_T5,Y_QH_PS,YAMT_PS_T1,YAMT_PS_T2,YAMT_PS_T3,YAMT_PS_T4,YAMT_PS_T5,Y_TOTAL_PE,YTOTAL_PE_PS_T1,YTOTAL_PE_PS_T2,YTOTAL_PE_PS_T3,YTOTAL_PE_PS_T4,YTOTAL_PE_PS_T5,YTOTAL_AMT_LD,YTOTAL_AMT_LD_AC,Y_TOTAL_QH,Y_TOTAL_QH_AC,Y_TOTAL_QH_AC_T1,Y_TOTAL_QH_AC_T2,Y_TOTAL_QH_AC_T3,Y_TOTAL_QH_AC_T4,Y_TOTAL_QH_AC_T5,Y_TOTAL_REPAY,Y_TOTAL_REPAY_T1,Y_TOTAL_REPAY_T2,Y_TOTAL_REPAY_T3,Y_TOTAL_REPAY_T4,Y_TOTAL_REPAY_T5,Y_TOTAL_REPAY_AC,Y_TOTAL_REPAY_AC_T1,Y_TOTAL_REPAY_AC_T2,Y_TOTAL_REPAY_AC_T3,Y_TOTAL_REPAY_AC_T4,Y_TOTAL_REPAY_AC_T5,Y_DUNO_CL,CUS_NAME,Y_EARLY_RPY,Y_EARLY_RPY_DATE,YWK_BAL_43,Y_CATEGORY,PRODUCT_TYPE,Y_SUB_NAME,Y_DRAWDOWN_NET_AMT,Y_RFD_AMT
2021-05-23 00:00:00.0000000,JOB_STG_T24_SAOKE_DEBT_COLLECTION_D01,2021-05-24 06:37:50.0000000,2021-05-23 00:00:00.0000000,2021-05-23 00:00:00.0000000,1000320060317241,LD2018102258,1059130,10126023346,180,0,NULL,NULL,NULL,NULL,NULL,27,11,2021-06-11 00:00:00.0000000,986522,687360,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1685882,1685702,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1673882,1673882,1673882,1673882,1673882,1673882,1673702,1673702,1673702,1673702,1673702,1673702,21873268,LE CONG TUOC,NULL,NULL,NULL,21061,CS,CS SY A 37,30000000.00,NULL


Nhìn vào câu query ở trên, ta có thể sẽ tự hỏi: Tại sao lại phải đưa toàn bộ cả câu `SELECT` của SQL Server vào nháy đơn và sử dụng `EXEC`, thay vì viết như ở dưới:

In [17]:
declare @y_sel_date varchar(12) = '2021-05-23'
SELECT * FROM OPENQUERY(ORASRV, 'SELECT * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION  where Y_SEL_DATE ='''+@y_sel_date+'''')

: Msg 102, Level 15, State 1, Line 2
Incorrect syntax near '+'.

Total execution time: 00:00:00.002

In [16]:
declare @y_sel_date varchar(12) = '2021-05-23'
declare @openquery varchar(4000) = 'SELECT * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION  where Y_SEL_DATE ='''+@y_sel_date+''''
select * from openquery(orasrv, @openquery)

: Msg 102, Level 15, State 1, Line 3
Incorrect syntax near '@openquery'.

Total execution time: 00:00:00.002

Cả 2 câu query ở trên đều gặp lỗi **Incorrect syntax**; lý do là bởi `OPENQUERY` chỉ nhận 2 tham số:

1. Tên Linked Server
2. Câu lệnh để gửi sang Oracle ở dạng **string** (text), tức là nằm trong nháy đơn

Tức là với tham số thứ 2, `OPENQUERY` sẽ chỉ nhận tham số có kiểu **string** và không nhận biến hay bất kỳ kiểu nào khác.

#### 2.3.2. Tối ưu hóa truy vấn dữ liệu khi sử dụng OPENQUERY

Khi sử dụng `OPENQUERY`, ta có 2 cách để đưa ra các điều kiện: 

- Ở bên ngoài OPENQUERY (tức là ở câu `SELECT` phía SQL Server)
- Ở bên trong OPENQUERY (tức là ở câu `SELECT` phía Oracle, nằm ở tham số thứ 2 của `OPENQUERY`, trong nháy đơn)

Xét 2 câu truy vấn sau:

In [21]:
-- Query 1
SELECT top 1 * FROM 
OPENQUERY(ORASRV ,'Select * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION  where Y_SEL_DATE = ''2021-05-23''' )

(1 row affected)

Total execution time: 00:00:00.021

COB_DT,UPD_BY,UPD_TIME,RUN_DATE,Y_SEL_DATE,Y_CONTRACT_REF,LD_ID,CUS_ID,YID_AC,YWK_BAL,Y_SONGAY_QH,Y_SONGAY_QH_T1,Y_SONGAY_QH_T2,Y_SONGAY_QH_T3,Y_SONGAY_QH_T4,Y_SONGAY_QH_T5,Y_KYHAN,Y_KYTRANO,Y_DATE_NEXT,Y_PR_NEXT,Y_IN_NEXT,Y_QH_PR,Y_QH_IN,Y_QH_PE,YAMT_PE_T1,YAMT_PE_T2,YAMT_PE_T3,YAMT_PE_T4,YAMT_PE_T5,Y_QH_PS,YAMT_PS_T1,YAMT_PS_T2,YAMT_PS_T3,YAMT_PS_T4,YAMT_PS_T5,Y_TOTAL_PE,YTOTAL_PE_PS_T1,YTOTAL_PE_PS_T2,YTOTAL_PE_PS_T3,YTOTAL_PE_PS_T4,YTOTAL_PE_PS_T5,YTOTAL_AMT_LD,YTOTAL_AMT_LD_AC,Y_TOTAL_QH,Y_TOTAL_QH_AC,Y_TOTAL_QH_AC_T1,Y_TOTAL_QH_AC_T2,Y_TOTAL_QH_AC_T3,Y_TOTAL_QH_AC_T4,Y_TOTAL_QH_AC_T5,Y_TOTAL_REPAY,Y_TOTAL_REPAY_T1,Y_TOTAL_REPAY_T2,Y_TOTAL_REPAY_T3,Y_TOTAL_REPAY_T4,Y_TOTAL_REPAY_T5,Y_TOTAL_REPAY_AC,Y_TOTAL_REPAY_AC_T1,Y_TOTAL_REPAY_AC_T2,Y_TOTAL_REPAY_AC_T3,Y_TOTAL_REPAY_AC_T4,Y_TOTAL_REPAY_AC_T5,Y_DUNO_CL,CUS_NAME,Y_EARLY_RPY,Y_EARLY_RPY_DATE,YWK_BAL_43,Y_CATEGORY,PRODUCT_TYPE,Y_SUB_NAME,Y_DRAWDOWN_NET_AMT,Y_RFD_AMT
2021-05-23 00:00:00.0000000,JOB_STG_T24_SAOKE_DEBT_COLLECTION_D01,2021-05-24 06:37:50.0000000,2021-05-23 00:00:00.0000000,2021-05-23 00:00:00.0000000,1000320060317241,LD2018102258,1059130,10126023346,180,0,NULL,NULL,NULL,NULL,NULL,27,11,2021-06-11 00:00:00.0000000,986522,687360,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1685882,1685702,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1673882,1673882,1673882,1673882,1673882,1673882,1673702,1673702,1673702,1673702,1673702,1673702,21873268,LE CONG TUOC,NULL,NULL,NULL,21061,CS,CS SY A 37,30000000.00,NULL


In [22]:
-- Query 2
-- Không nên chạy vì tốn thời gian
SELECT top 1 * FROM 
OPENQUERY(ORASRV ,'Select * from IT_USER.STG_T24_SAOKE_DEBT_COLLECTION ')
WHERE Y_SEL_DATE = '2021-05-23'

: Query failed: The operation was canceled.

Query was canceled by user

Total execution time: 00:05:31.134

Có thể thấy tốc độ truy vấn của Query 2 chậm hơn hẳn Query 1. Lý do là bởi trong Query 2, `OPENQUERY` đã lấy toàn bộ bảng `IT_USER.STG_T24_SAOKE_DEBT_COLLECTION` sang bên SQL Server và sau đó mới áp đặt điều kiện `Y_SEL_DATE = '2021-05-23'`. Ngược lại, ở Query 1, điều kiện `Y_SEL_DATE = '2021-05-23'` được áp đặt ngay từ phía ORACLE rồi sau đó mới lấy sang phía SQL Server.

Trên lý thuyết, ta nên đưa hết tất cả các điều kiện cũng như giới hạn số lượng cột cần `SELECT` ngày từ phía query của ORACLE. Điều này giúp lượng dữ liệu mang sang SQL Server nhỏ hơn, và toàn bộ tác vụ xử dữ liệu sẽ nằm ở Oracle.

Tuy nhiên trên thực tế, việc đưa nhiều điều kiện hay các tác vụ xử lý dữ liệu vào câu query ở phía Oracle sẽ làm cho việc viết query phức tạp hơn nếu mỗi điều kiện lại tương ứng với một biến phải khai báo. Như vậy, ta có thể cân đối giữa việc xử lý dữ liệu ở phía ORACLE hoặc đưa về SQL Server rồi mới xử lý, nhằm đảm bảo cả về tốc độ truy vấn và tính dễ hiểu của câu query cuối cùng.

#### 2.3.3. Sử dụng vòng lặp để kiểm tra dữ liệu ở ORACLE

Do dữ liệu ở ORACLE nằm ở máy chủ `172.17.101.148/MOMPDB` được động bộ hóa chậm 1 ngày (T-1) vào khoảng 9h sáng hàng này nên ta cần có cách để kiểm tra xem dữ liệu đã có chưa trước khi thực hiện query/chạy job ở phía SQL Server.

Logic một vòng lặp đơn giản để kiểm tra dữ liệu có thể được thiết kế như sau:

- Kiểm tra dữ liệu lần đầu ở Oracle, nếu có dữ liệu sẽ trả về giá trị 1, nếu chưa sẽ trả về giá trị 0. Gọi giá trị trả về là `table_status`
- Bắt đầu vòng lặp với điều kiện: Nếu `table_status` == 0, tiếp tục kiểm tra dữ liệu và cập nhật `table_status` sau mỗi lần kiểm tra
- Nếu `table_status` == 1, vòng lặp kết thúc và chuyển sang kéo dữ liệu từ ORACLE về

In [ ]:
drop table if exists ##table_status

declare @plsql varchar(8000) = 'select * from oracle_table'
declare @checking_plsql varchar(8000) = 'select case when exists(' + @plsql +') then 1 else 0 end as table_status from dual'
declare @checking_tsql varchar(8000) = 'select * into ##table_status_trghm from openquery(orasrv, '''+@checking_plsql+''')'
declare @insert_tsql varchar(8000) = 'insert into sql_server_table select * from openquery(orasrv, '''+@plsql+''')'
declare @first_exec_timestamp datetime = getdate()

exec(@checking_tsql)
while (select table_status from ##table_status) = 0
begin
    exec(@checking_tsql)
    WAITFOR DELAY '00:00:15' -- chờ 15s trước khi kiểm tra lại dữ liệu để tránh việc quá tại cho server
    declare @table_status int = (select table_status from ##table_status)
    print(@table_status)
    if datediff(mi, @first_exec_timestamp, getdate()) > 60 -- nếu việc kiểm tra dữ liệu kéo dài quá 60 phút, dừng lại việc kiểm tra
    begin
        break
    end
end;

-- select table_status from ##table_status

if (select table_status from ##table_status) = 1
begin
    WAITFOR DELAY '00:02:00' -- chờ 2 phút trước khi thực hiện kéo dữ liệu về để đảm bảo dữ liệu bên phía ORACLE đã có đủ
    exec(@insert_tsql)
end;


## 3. Một số cú pháp PL/SQL (Oracle) cần biết và cú pháp tương ứng ở SQL Server

1. Thay thế `TOP N` trong SQL Server


In [ ]:
select *
from  table_name
where  ROWNUM  <=  N;

2. Kiểm tra ngày hiện tại

In [ ]:
select SYSDATE from dial;

3. Thay thế `CONVERT` trong SQL Server

In [ ]:
SELECT TO_DATE('20201201', "YYYYMMDD") FROM DIAL;
SELECT TO_CHAR(SYSDATE, "YYYYMMDD") FROM DIAL;

4. `FROM DIAL`

Trong ORACLE bắt buộc mọi câu SELECT phải có FROM TABLE đi kèm -> với các câu select tương ứng trong SQL Server, thêm vào FROM DUAL.

- Trong Oracle:

In [ ]:
select SYSDATE from dial;

- Trong SQL Server:

In [ ]:
select GETDATE()

5. Thay thế `ISNULL()` trong SQL Server

In [ ]:
SELECT NVL(col_name, 'subtitute_value')
FROM table_name;

6. Thay thế hàm `LEN()` trong SQL Server

In [ ]:
SELECT LENGTH ('string') FROM DUAL;

7. Thay thế `LEFT`, `RIGHT` và `SUBSTRING` trong SQL Server

In [ ]:
SUBSTR( string, start_position [, length ] )

8. Thay thế `CEILING` và `FLOOR` trong SQL server

In [ ]:
-- thay thế CEILING
SELECT CEIL(5.5555) FROM DIAL;
-- thay thế fLOOR
SELECT FLOOR(5.5555) FROM DIAL;

9. THAY THẾ các hàm `DAY`, `MONTH` và `YEAR` và `DATEPART`

In [ ]:
SELECT EXTRACT(DAY FROM SYSDATE) as DAY;
SELECT EXTRACT(MONTH FROM SYSDATE) as MONTH;
SELECT EXTRACT(YEAR FROM SYSDATE) as YEAR;